# 📋 YOLOv12 Checkbox Detection Training

This notebook trains a YOLOv12 model for checkbox detection in PDF documents.

**Setup:** Make sure you're using GPU runtime (A100 recommended for Colab Pro)

## 🔧 Step 1: Setup Environment

In [ ]:
# Clone your project repository (replace with your actual repo)
!git clone https://github.com/yourusername/OCR_checkbox.git /content/checkbox_detection

# Or upload the project files manually
# For now, we'll create the structure manually

In [ ]:
# Upload the colab setup script
from google.colab import files
import os

# Create colab directory
os.makedirs('/content/checkbox_detection/colab', exist_ok=True)

print("📁 Upload your colab/setup_colab_environment.py file:")
uploaded = files.upload()

# Move uploaded file to correct location
for filename in uploaded.keys():
    if filename.endswith('.py'):
        os.rename(filename, f'/content/checkbox_detection/colab/{filename}')
        print(f"✅ Uploaded {filename}")

In [ ]:
# Run the setup script
%run /content/checkbox_detection/colab/setup_colab_environment.py

## 📊 Step 2: Upload Your Dataset

In [ ]:
# Option 1: Upload dataset from your computer
from google.colab import files
import zipfile
import os

print("📁 Upload your prepared dataset (zip file):")
print("Expected structure: data/processed/train/, data/processed/val/, data/processed/test/")

uploaded = files.upload()

# Extract dataset
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        print(f"📦 Extracting {filename}...")
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('/content/checkbox_detection/')
        os.remove(filename)  # Clean up
        print("✅ Dataset extracted!")

In [ ]:
# Option 2: Copy from Google Drive (if you uploaded there)
# Uncomment and modify the path if you stored data in Drive

# !cp -r "/content/drive/MyDrive/checkbox_detection_data/*" "/content/checkbox_detection/data/"

In [ ]:
# Verify dataset structure
import os
from pathlib import Path

data_dir = Path('/content/checkbox_detection/data/processed')
print("📊 Dataset verification:")

for split in ['train', 'val', 'test']:
    images_dir = data_dir / split / 'images'
    labels_dir = data_dir / split / 'labels'
    
    if images_dir.exists() and labels_dir.exists():
        num_images = len(list(images_dir.glob('*.png'))) + len(list(images_dir.glob('*.jpg')))
        num_labels = len(list(labels_dir.glob('*.txt')))
        print(f"✅ {split}: {num_images} images, {num_labels} labels")
    else:
        print(f"❌ {split}: Missing directories")

# Check for data.yaml
data_yaml = data_dir / 'data.yaml'
if data_yaml.exists():
    print(f"✅ Found data.yaml configuration")
    # Show content
    with open(data_yaml, 'r') as f:
        print("Content:")
        print(f.read())
else:
    print(f"❌ data.yaml not found - will create one")

## 🚀 Step 3: Train YOLOv12 Model

In [ ]:
# Test YOLOv12 availability first
from ultralytics import YOLO
import torch

print("🧪 Testing YOLOv12 availability...")

# Check GPU
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"🎮 GPU: {gpu_name} ({gpu_memory:.1f} GB)")
else:
    print("❌ No GPU detected!")

# Test YOLOv12
try:
    model = YOLO('yolo12n.pt')
    print("✅ YOLOv12 loaded successfully!")
    print(f"📊 Parameters: {sum(p.numel() for p in model.model.parameters()):,}")
except Exception as e:
    print(f"❌ YOLOv12 failed to load: {e}")

In [ ]:
# Upload training script
from google.colab import files
import os

print("📁 Upload your colab/train_yolo12_colab.py file:")
uploaded = files.upload()

# Move to correct location
for filename in uploaded.keys():
    if filename.endswith('.py'):
        os.rename(filename, f'/content/checkbox_detection/colab/{filename}')
        print(f"✅ Uploaded {filename}")

In [ ]:
# Start training!
%cd /content/checkbox_detection

# Run the training script
!python colab/train_yolo12_colab.py

## 📊 Step 4: Monitor Training Progress

In [ ]:
# View training plots
import matplotlib.pyplot as plt
from IPython.display import Image, display
import glob

# Find training results
results_dirs = glob.glob('/content/checkbox_detection/models/yolo12_training_*/results.png')

if results_dirs:
    print("📊 Training Results:")
    for results_file in results_dirs[-3:]:  # Show last 3 runs
        print(f"📈 {results_file}")
        display(Image(results_file))
else:
    print("📊 No training results found yet - training may still be in progress")

# Show confusion matrix if available
confusion_matrices = glob.glob('/content/checkbox_detection/models/yolo12_training_*/confusion_matrix.png')
if confusion_matrices:
    print("🎯 Confusion Matrix:")
    display(Image(confusion_matrices[-1]))  # Show latest

In [ ]:
# Check final metrics
import json
from pathlib import Path

# Look for training metrics in Drive backup
metrics_files = list(Path('/content/drive/MyDrive/checkbox_detection_results').glob('*/training_metrics.json'))

if metrics_files:
    latest_metrics = metrics_files[-1]  # Get most recent
    
    with open(latest_metrics, 'r') as f:
        metrics = json.load(f)
    
    print("🎉 Final Training Metrics:")
    print(f"📊 mAP@0.5: {metrics.get('mAP50', 'N/A'):.4f}")
    print(f"📊 mAP@0.5:0.95: {metrics.get('mAP50_95', 'N/A'):.4f}")
    print(f"📊 Precision: {metrics.get('precision', 'N/A'):.4f}")
    print(f"📊 Recall: {metrics.get('recall', 'N/A'):.4f}")
    print(f"⏱️ Training time: {metrics.get('training_time_hours', 'N/A'):.1f} hours")
    
    # Check if target achieved
    if metrics.get('mAP50', 0) > 0.85:
        print("🎯 🎉 TARGET ACHIEVED: mAP@0.5 > 0.85! 🎉")
    else:
        print(f"🎯 Target: mAP@0.5 > 0.85 (Current: {metrics.get('mAP50', 0):.4f})")
else:
    print("📊 No metrics found - training may still be in progress")

## 📥 Step 5: Download Results

In [ ]:
# Prepare download package
import shutil
from pathlib import Path
from google.colab import files

print("📦 Preparing results for download...")

# Create download directory
download_dir = Path('/content/download_package')
download_dir.mkdir(exist_ok=True)

# Copy best model weights
models_dir = Path('/content/checkbox_detection/models')
for model_dir in models_dir.glob('yolo12_training_*'):
    weights_dir = model_dir / 'weights'
    if (weights_dir / 'best.pt').exists():
        target_dir = download_dir / f'model_{model_dir.name}'
        target_dir.mkdir(exist_ok=True)
        
        # Copy model files
        shutil.copy2(weights_dir / 'best.pt', target_dir)
        if (weights_dir / 'last.pt').exists():
            shutil.copy2(weights_dir / 'last.pt', target_dir)
        
        print(f"✅ Copied model: {model_dir.name}")

# Copy training plots
for plot in models_dir.glob('*/results.png'):
    shutil.copy2(plot, download_dir / f'{plot.parent.name}_results.png')

# Copy confusion matrices
for cm in models_dir.glob('*/confusion_matrix.png'):
    shutil.copy2(cm, download_dir / f'{cm.parent.name}_confusion_matrix.png')

# Create zip file
shutil.make_archive('/content/checkbox_detection_results', 'zip', download_dir)

print("✅ Results package ready!")
print(f"📦 Package size: {Path('/content/checkbox_detection_results.zip').stat().st_size / 1e6:.1f} MB")

# Download
files.download('/content/checkbox_detection_results.zip')
print("📥 Download started!")

## 🧪 Step 6: Test Trained Model (Optional)

In [ ]:
# Quick test of the trained model
from ultralytics import YOLO
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Load the best trained model
model_path = None
for model_dir in Path('/content/checkbox_detection/models').glob('yolo12_training_*'):
    best_path = model_dir / 'weights' / 'best.pt'
    if best_path.exists():
        model_path = best_path
        break

if model_path:
    print(f"📥 Loading trained model: {model_path}")
    model = YOLO(str(model_path))
    
    # Create a test image with synthetic checkboxes
    img = np.ones((400, 600, 3), dtype=np.uint8) * 255
    
    # Draw test checkboxes
    checkboxes = [(50, 50, 30, 30), (200, 100, 25, 25), (350, 150, 35, 35)]
    for x, y, w, h in checkboxes:
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 0), 2)
        # Add checkmark to first one
        if x == 50:
            cv2.line(img, (x + 5, y + 15), (x + 12, y + 22), (0, 0, 0), 2)
            cv2.line(img, (x + 12, y + 22), (x + 25, y + 8), (0, 0, 0), 2)
    
    # Run detection
    results = model.predict(img, conf=0.25, verbose=False)
    
    # Visualize results
    if results[0].boxes is not None and len(results[0].boxes) > 0:
        print(f"🎯 Detected {len(results[0].boxes)} checkboxes!")
        
        # Draw detections
        for box in results[0].boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
            conf = box.conf[0].cpu().numpy()
            
            # Draw bounding box
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, f'{conf:.2f}', (x1, y1-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        
        # Display
        plt.figure(figsize=(10, 6))
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.title('Checkbox Detection Test')
        plt.axis('off')
        plt.show()
        
    else:
        print("🤷 No checkboxes detected - model may need more training")
        
else:
    print("❌ No trained model found")